In [1]:
# show result of last expression even if it was an assignment
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

# Step 1: The Station List

In [2]:
import json
from pathlib import Path

# JSON data from 
# https://layer.bicyclesharing.net/map/v1/fgb/map-inventory
map_json_path = Path("./map.json")
with map_json_path.open() as f:
    map_json = json.load(f)

In [3]:
from typing import NamedTuple

class Station(NamedTuple):
    lat: float
    lon: float
    name: str
    id: str
       
    @property
    def lonlat(self):
        return self.lon, self.lat
       
    @property
    def latlon(self):
        return self.lat, self.lon
        
    def __repr__(self):
        return f"{self.lat:9.4f} {self.lon:9.4f} {self.id:10} {self.name[:30]}"

In [4]:
stations = [
    Station(
        lat=feature["geometry"]["coordinates"][1],
        lon=feature["geometry"]["coordinates"][0],
        name=feature["properties"]["station"]["name"],
        id=feature["properties"]["station"]["terminal"],
    ) 
    for feature in map_json["features"] 
    if feature["properties"].get("station", {}).get("installed")
]

[  37.7864 -122.4049 SF-G27     Powell St BART Station (Market,
   37.7859 -122.4089 SF-G26     Cyril Magnin St at Ellis St,
   37.7839 -122.4084 SF-H26     Powell St BART Station (Market,
   37.8048 -122.4032 SF-A27     The Embarcadero at Sansome St,
   37.8046 -122.2717 OK-L5      Frank H Ogawa Plaza,
   37.8000 -122.3985 SF-C28-1   The Embarcadero at Vallejo St,
   37.7986 -122.4009 SF-C28-2   Broadway at Battery St,
   37.7954 -122.4048 SF-D27     Washington St at Kearny St,
   37.7973 -122.3984 SF-D28     Davis St at Jackson St,
   37.7942 -122.4029 SF-E27     Commercial St at Montgomery St,
   37.7950 -122.4000 SF-E28     Clay St at Battery St,
   37.7954 -122.3942 SF-E29-1   San Francisco Ferry Building (,
   37.7941 -122.3944 SF-E29-2   Steuart St at Market St,
   37.7923 -122.3971 SF-E29-3   Embarcadero BART Station (Beal,
   37.8502 -122.2602 OK-A2      Telegraph Ave at Alcatraz Ave,
   37.7890 -122.4035 SF-F27     Post St at Kearny St,
   37.7913 -122.3991 SF-F28-1   Mechani

In [5]:
from collections import Counter
prefixes = Counter([st.id.split("-")[0] for st in stations])
prefixes.most_common()

[('SF', 128),
 ('OK', 79),
 ('SJ', 44),
 ('BK', 35),
 ('EM', 10),
 ('SF J29', 1),
 ('SF I29', 1)]

In [6]:
stations = [st for st in stations if st.id[:2] in {"OK", "BK", "EM"}]
len(stations)

124

# Step 2: Folium for Mapping

In [7]:
import folium

def basemap(stations):
    center_lat = 0.5 * (min(st.lat for st in stations) + max(st.lat for st in stations))
    center_lon = 0.5 * (min(st.lon for st in stations) + max(st.lon for st in stations))
    
    mymap = folium.Map(
        location=[center_lat, center_lon], 
        zoom_start=12,
        tiles='Stamen Terrain',
    )

    for st in stations:
        folium.Circle(
            location=st.latlon, 
            radius=50,
            popup=f"<strong>{st.id}</strong> {st.name}",
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1.0,
        ).add_to(mymap)
        
    return mymap

In [8]:
basemap(stations)

# Step 3: Traveling Salesman with OR Tools

In [9]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

manager = pywrapcp.RoutingIndexManager(
    len(stations),  # number of stations
    1,              # number of vehicles
    0,              # index of start/finish station
)
routing = pywrapcp.RoutingModel(manager)

<ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x10d098510> >

In [10]:
from geopy.distance import distance

def as_the_bird_flies(from_station: Station, to_station: Station):
    """Returns distance between two stations in whole meters"""
    dist = distance(from_station.latlon, to_station.latlon)
    return int(dist.meters)  

def cost_function(from_index, to_index):
    """Cost function for OR Tools"""
    from_station = stations[manager.IndexToNode(from_index)]
    to_station = stations[manager.IndexToNode(to_index)]
    return as_the_bird_flies(from_station, to_station)

In [11]:
# not a particularly Pythonic API...
callback_ptr = routing.RegisterTransitCallback(cost_function)
routing.SetArcCostEvaluatorOfAllVehicles(callback_ptr)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.time_limit.seconds = 120

# This lines will run for a few minutes!
assignment = routing.SolveWithParameters(search_parameters)

total_dist = assignment.ObjectiveValue()
print(f"Distance: {total_dist/1000:.3} km ({total_dist/1609:.3} mi)")

Distance: 64.7 km (40.2 mi)


In [12]:
def route_from_ortools_solution(assignment, routing, manager):
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route.append((
            stations[manager.IndexToNode(previous_index)],
            stations[manager.IndexToNode(index)]
        ))
    return route

route = route_from_ortools_solution(assignment, routing, manager)
routemap = basemap(stations)

for st1, st2 in route:
    folium.PolyLine([st1.latlon, st2.latlon]).add_to(routemap)

routemap

# Step 4: Real cycling directions from Mapbox

In [13]:
import mapbox

service = mapbox.Directions(access_token="secret")

In [14]:
class Route(NamedTuple):
    distance: float
    duration: float
    polyline: folium.PolyLine


def get_route_from_mapbox(from_station, to_station):
    """Fetch directions from Mapbox API and turn into Folium polyline"""
    resp = service.directions(
        [from_station.lonlat, to_station.lonlat],
        profile="mapbox/cycling",
        geometries="polyline",
    ).geojson()
    
    # flip coordinates from lon/lat to lat/lon
    coordinates = resp["features"][0]["geometry"]["coordinates"]
    flipped = [(lat, lon) for (lon, lat) in coordinates]
    
    return Route(
        distance=resp["features"][0]["properties"]["distance"],
        duration=resp["features"][0]["properties"]["duration"],
        polyline=folium.PolyLine(flipped),
    )

In [16]:
import itertools

frommapbox = {}
for from_station, to_station in itertools.combinations(stations, 2):
    
    # If the straight line distance is less than one and a half
    # miles, then get the cycling distance from the Mapbox API.
    straight_line = as_the_bird_flies(from_station, to_station)
    if straight_line < 1.5 * 1609:
        
        # Both directions because the route might be different
        # due to one-way streets, etc.
        frommapbox[(from_station, to_station)] = \
            get_route_from_mapbox(from_station, to_station)
        frommapbox[(to_station, from_station)] = \
            get_route_from_mapbox(to_station, from_station)

In [17]:
debugmap2 = basemap(stations)
frommapbox[(stations[4], stations[10])].polyline.add_to(debugmap2)
debugmap2

# Step 5: Traveling Salesman, take 2

In [18]:
def cost_function_2(from_index, to_index):
    """Cost function for OR Tools"""
    from_station = stations[manager.IndexToNode(from_index)]
    to_station = stations[manager.IndexToNode(to_index)]
    
    try:
        route = frommapbox[(from_station, to_station)]
        return int(route.duration)  # in seconds
    
    except KeyError:
        # overestimate duration for routes we don't
        # have directions for (really want to avoid them)
        return as_the_bird_flies(from_station, to_station) * 5

In [19]:
manager_2 = pywrapcp.RoutingIndexManager(len(stations), 1, 0)
routing_2 = pywrapcp.RoutingModel(manager_2)
callback_ptr_2 = routing_2.RegisterTransitCallback(cost_function_2)
routing_2.SetArcCostEvaluatorOfAllVehicles(callback_ptr_2)
search_parameters_2 = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_2.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
assignment_2 = routing_2.SolveWithParameters(search_parameters_2)

total_dist = assignment_2.ObjectiveValue()
print(f"Duration: {total_dist/3600:.3} hrs")

Duration: 6.62 hrs


In [20]:
route_2 = route_from_ortools_solution(assignment_2, routing_2, manager_2)
routemap_2 = basemap(stations)

for st1, st2 in route_2:
    try:
        polyline = frommapbox[(st1, st2)].polyline
        polyline.add_to(routemap_2)
    except KeyError:
        folium.PolyLine([st1.latlon, st2.latlon], color="pink").add_to(routemap_2)

routemap_2

Modifying the cost function to penalize rides with durations under 2 minutes a little bit. This cost function considers 2 minutes (120 seconds) ride length the optimum. 90 second ride length gets the same cost as 150 seconds, 60 second ride length gets the same cost as 180 seconds.

In [21]:
def cost_function_3(from_index, to_index):
    """Cost function for OR Tools"""
    from_station = stations[manager.IndexToNode(from_index)]
    to_station = stations[manager.IndexToNode(to_index)]
    
    try:
        route = frommapbox[(from_station, to_station)]
        return int(abs(120 - route.duration))  # seconds
    
    except KeyError:
        # grossly overestimate duration for routes we don't
        # have directions for (really want to avoid them)
        return as_the_bird_flies(from_station, to_station) * 5

In [22]:
manager_3 = pywrapcp.RoutingIndexManager(len(stations), 1, 0)
routing_3 = pywrapcp.RoutingModel(manager_3)
callback_ptr_3 = routing_3.RegisterTransitCallback(cost_function_3)
routing_3.SetArcCostEvaluatorOfAllVehicles(callback_ptr_3)
search_parameters_3 = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_3.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
assignment_3 = routing_3.SolveWithParameters(search_parameters_3)

<ortools.constraint_solver.pywrapcp.Assignment; proxy of <Swig Object of type 'operations_research::Assignment *' at 0x1103e51b0> >

In [23]:
route_3 = route_from_ortools_solution(assignment_3, routing_3, manager_3)
routemap_3 = basemap(stations)

for st1, st2 in route_3:
    polyline = frommapbox[(st1, st2)].polyline
    polyline.add_to(routemap_3)

routemap_3